In [2]:
import wandb
wandb.login(key = "eb9574fa5b11da36782604ea27df8bf1989ddefd")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: megh_m (megh_m-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
import os
os.environ["WANDB_SILENT"] = "false"
os.environ["WANDB_START_METHOD"] = "thread"
os.environ["WANDB_API_KEY"] = "eb9574fa5b11da36782604ea27df8bf1989ddefd"

In [25]:
import numpy as np
import torch
import torch.nn as nn
import random

# Utils

In [23]:
class CharEmbed(nn.Module):
    def __init__(self, input_dim, embed_dim):
        super(CharEmbed, self).__init__()
        self.embed = nn.Embedding(input_dim, embed_dim)
    
    def forward(self, input_seq):
        return self.embed(input_seq)

class EncoderRNN(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, n_layers=1, 
                 cell_type='GRU', dropout=0.0, bidirectional=False):
        super(EncoderRNN, self).__init__()
        self.embed = nn.Embedding(input_dim, embed_dim)
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.cell_type = cell_type
        self.bidirectional = bidirectional #to allow forward and backward time step data processing
        # Cell type options GRU, LSTM & vanilla RNN
        if cell_type == 'GRU':
            self.rnn = nn.GRU(embed_dim, hidden_dim, n_layers, dropout=dropout if n_layers > 1 else 0, bidirectional=bidirectional)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout if n_layers > 1 else 0, bidirectional=bidirectional)
        else: 
            self.rnn = nn.RNN(embed_dim, hidden_dim, n_layers, dropout=dropout if n_layers > 1 else 0, bidirectional=bidirectional)
    
    def forward(self, input_seq, input_lengths, hidden=None):
        # Sort sequences by length
        input_lengths, sort_idx = torch.sort(input_lengths, descending=True)
        input_seq = input_seq[:, sort_idx]  # (seq_len, batch_size, ...)
        
        # Convert to embeddings
        embedded = self.embed(input_seq)
        
        # Pack with enforce_sorted=False
        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, 
            input_lengths.cpu(), 
            enforce_sorted=False
        )
        
        # Forward pass
        outputs, hidden = self.rnn(packed, hidden)
        
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        
        # Restore original order
        _, unsort_idx = torch.sort(sort_idx)
        outputs = outputs[:, unsort_idx]
        
        # Handle LSTM hidden/cell states
        if isinstance(hidden, tuple):
            hidden = (
                hidden[0][:, unsort_idx],  # Hidden state
                hidden[1][:, unsort_idx]   # Cell state
            )
        else:  # For GRU/RNN
            hidden = hidden[:, unsort_idx]
        
        return outputs, hidden

class DecoderRNN(nn.Module): #Basically similar to the encoder, will have a softmax to predict next char
    def __init__(self, output_dim, embed_dim, hidden_dim, n_layers=1, cell_type='GRU', dropout=0.0):
        super(DecoderRNN, self).__init__()
        self.embed = nn.Embedding(output_dim, embed_dim)
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.cell_type = cell_type
        if cell_type == 'GRU':
            self.rnn = nn.GRU(embed_dim, hidden_dim, n_layers, dropout=dropout if n_layers > 1 else 0)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout if n_layers > 1 else 0)
        else:
            self.rnn = nn.RNN(embed_dim, hidden_dim, n_layers, dropout=dropout if n_layers > 1 else 0)
        
        self.out = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        # Get embedding of current input character
        embedded = self.embed(input).unsqueeze(0)
        
        # Forward pass through decoder
        output, hidden = self.rnn(embedded, hidden)
        
        # Predict next character probabilities
        output = self.softmax(self.out(output.squeeze(0)))
        
        return output, hidden


In [16]:
class Seq2Seq(nn.Module): #Flexible enough to use different encoders other than the ones we define
    def __init__(self, input_dim, output_dim, embed_dim, 
                 hidden_dim, num_layers, cell_type, dropout, device):
        #super().__init__()
        super(Seq2Seq, self).__init__()
        self.device = device
            # Internal encoder creation
        self.encoder = EncoderRNN(
            input_dim=input_dim,
            embed_dim=embed_dim,
            hidden_dim=hidden_dim,
            n_layers=num_layers,
            cell_type=cell_type,
            dropout=dropout
        )
        
        # Internal decoder creation
        self.decoder = DecoderRNN(
            output_dim=output_dim,
            embed_dim=embed_dim,
            hidden_dim=hidden_dim,
            n_layers=num_layers,
            cell_type=cell_type,
            dropout=dropout
        )
        
        self.device = device 
    def forward(self, src, src_len, trg, teacher_forcing_ratio=0.5):
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        # Tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        # Last hidden state of the encoder
        encoder_outputs, hidden = self.encoder(src, src_len)
        
        # First input to the decoder is the <go> token
        input = trg[0,:]
        
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden)
            outputs[t] = output
            
            # To Decide if we're going to use teacher forcing or not as needed
            teacher_force = random.random() < teacher_forcing_ratio
            
            # Get the highest predicted token from our predictions
            top1 = output.argmax(1)
            
            # If we use teacher forcing, we have to use actual next token as next input
            # If not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs


# Dataset Loading & Preprocessing

In [13]:
import os
import tarfile
import requests
import pandas as pd
from io import BytesIO
from collections import defaultdict
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import wandb

# Dataset Configuration
DATASET_URL = "https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar"
DATA_DIR = "./dakshina_dataset"
HI_LEXICON_DIR = os.path.join(DATA_DIR,"dakshina_dataset_v1.0", "hi", "lexicons") #For Hindi (Chosen Language)

def download_and_extract_dataset(): #Scripted Dataset Download
    if not os.path.exists(DATA_DIR):
        print("Downloading dataset...")
        response = requests.get(DATASET_URL)
        file = tarfile.open(fileobj=BytesIO(response.content))
        file.extractall(DATA_DIR)
        print("Dataset extracted successfully")

class TransliterationVocabulary: #Build Character Vocab and add go,stop, padding and unknown tokens
    def __init__(self):
        self.char2idx = defaultdict(lambda: len(self.char2idx))
        self.idx2char = {}
        self.special_tokens = ['<pad>', '<go>', '<stop>', '<unk>']
        
        # Initialize special tokens
        for token in self.special_tokens:
            self.char2idx[token]
        
        self.idx2char = {v: k for k, v in self.char2idx.items()}
    
    def add_word(self, word):
        #print(word) #for debugging
        for char in word:
            self.char2idx[char]
        self.idx2char = {v: k for k, v in self.char2idx.items()}

class TransliterationDataset(Dataset): #Dataset loader for Hindi
    def __init__(self, split='train'):
        self.split = split
        self.data = self._load_data()
        self.src_vocab = TransliterationVocabulary()
        self.trg_vocab = TransliterationVocabulary()
        
        # Build vocabularies
        for pair in self.data:
            self.src_vocab.add_word(pair[0])
            self.trg_vocab.add_word(pair[1])
    
    def _load_data(self):
        """Load data from TSV files and filter non-string entries"""
        file_map = {
            'train': 'hi.translit.sampled.train.tsv',
            'dev': 'hi.translit.sampled.dev.tsv',
            'test': 'hi.translit.sampled.test.tsv'
        }
        
        df = pd.read_csv(
            os.path.join(HI_LEXICON_DIR, file_map[self.split]),
            sep='\t', 
            header=None,
            names=['latin', 'devanagari'],
            dtype={'latin': str, 'devanagari': str}  # Force string type
        )
        
        # Filter out non-string entries and empty strings
        valid_entries = [
            (latin, devanagari) 
            for latin, devanagari in zip(df['latin'], df['devanagari'])
            if (isinstance(latin, str) and 
                isinstance(devanagari, str) and
                len(latin) > 0 and 
                len(devanagari) > 0)
        ]
        
        return valid_entries

    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        src, trg = self.data[idx]
        return (
            [self.src_vocab.char2idx['<go>']] + 
            [self.src_vocab.char2idx[c] for c in src] +
            [self.src_vocab.char2idx['<stop>']],
            [self.trg_vocab.char2idx['<go>']] + 
            [self.trg_vocab.char2idx[c] for c in trg] +
            [self.trg_vocab.char2idx['<stop>']]
        )

def collate_fn(batch): #Padding and Masking
    src_batch, trg_batch = zip(*batch)
    
    src_lens = torch.tensor([len(x) for x in src_batch])
    trg_lens = torch.tensor([len(x) for x in trg_batch])
    
    src_pad = pad_sequence(
        [torch.tensor(x) for x in src_batch],
        padding_value=0  # <pad> token index
    )
    
    trg_pad = pad_sequence(
        [torch.tensor(x) for x in trg_batch],
        padding_value=0  # <pad> token index
    )
    
    return src_pad, trg_pad, src_lens, trg_lens

def get_dataloaders(batch_size=64):
    """Create train, dev, test dataloaders"""
    download_and_extract_dataset()
    
    train_dataset = TransliterationDataset('train')
    dev_dataset = TransliterationDataset('dev')
    test_dataset = TransliterationDataset('test')
    
    return (
        DataLoader(train_dataset, batch_size=batch_size, 
                  shuffle=True, collate_fn=collate_fn),
        DataLoader(dev_dataset, batch_size=batch_size, 
                 collate_fn=collate_fn),
        DataLoader(test_dataset, batch_size=batch_size,
                 collate_fn=collate_fn),
        train_dataset.src_vocab,
        train_dataset.trg_vocab
    )


In [ ]:
def beam_search_decode(model, src, src_len, beam_size, max_len, device):
    # Encode source sequence
    encoder_outputs, encoder_hidden = model.encoder(src, src_len)
    
    # Initialize beam
    GO_token = model.decoder.vocab['<go>']
    beam = [BeamNode([SOS_token], encoder_hidden, 0.0)]
    
    for step in range(max_len):
        candidates = []
        for node in beam:
            if node.is_finished():
                candidates.append(node)
                continue
            
            # Prepare decoder input
            input_tensor = torch.LongTensor([node.tokens[-1]]).to(device)
            
            # Decoder step
            log_probs, hidden = model.decoder.beam_step(
                input_tensor, 
                node.hidden,
                encoder_outputs
            )
            
            # Expand candidates
            topk_probs, topk_ids = log_probs.topk(beam_size)
            for i in range(beam_size):
                token = topk_ids[0][i].item()
                score = node.score + topk_probs[0][i].item()
                new_node = BeamNode(
                    node.tokens + [token],
                    hidden,
                    score,
                    token == model.decoder.vocab['<stop>']
                )
                candidates.append(new_node)
        
        # Select top-k candidates
        candidates.sort(reverse=True)
        beam = candidates[:beam_size]
        
        # Early stopping if all beams finished
        if all(node.is_finished() for node in beam):
            break
    
    return beam[0].tokens[1:-1]  # Remove SOS and EOS


In [12]:
download_and_extract_dataset()

Dataset extracted successfully


In [42]:

df = pd.read_csv(
            os.path.join(HI_LEXICON_DIR,'hi.translit.sampled.train.tsv'),
            sep='\t',  #specifying seperator
            header=None,
            names=['devanagari','latin','syllables']
        )
list(zip(df['latin'], df['devanagari']))

44204

# Train Sweep

In [10]:
sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'val_acc', 'goal': 'maximize'},
    'parameters': {
        'embedding_size': {'values': [64, 128, 256]},
        'hidden_size': {'values': [128, 256, 512]},
        'num_layers': {'values': [1, 2, 3]},
        'cell_type': {'values': ['LSTM', 'GRU', 'RNN']},
        'dropout': {'values': [0.2, 0.3]},
        'learning_rate': {'values': [0.001, 0.0005, 0.0001]},
        'batch_size': {'values': [32, 64, 128]}
    }
}

In [15]:
def train(config=None):
    with wandb.init(project="DA6401_A3",settings=wandb.Settings(start_method="thread",_disable_stats=True), config = config) as run:
        config = run.config
        
        # Get dataloaders and vocabularies
        train_loader, dev_loader, _, src_vocab, trg_vocab = get_dataloaders(
            batch_size=config.batch_size
        )
        
        # Initialize model
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        model = Seq2Seq(
            input_dim=len(src_vocab.char2idx),
            output_dim=len(trg_vocab.char2idx),
            embed_dim=config.embedding_size,
            hidden_dim=config.hidden_size,
            num_layers=config.num_layers,
            cell_type=config.cell_type,
            dropout=config.dropout,
            device=device
        ).to(device)
        
        # Training setup
        optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
        criterion = torch.nn.CrossEntropyLoss(ignore_index=0)
        
        # Training loop
        for epoch in range(10):  # Fixed epoch count for sweep
            model.train()
            total_loss = 0
            
            for src, trg, src_lens, trg_lens in train_loader:
                src = src.to(device)
                trg = trg.to(device)
                
                optimizer.zero_grad()
                output = model(src, src_lens, trg)
                
                # Calculate loss
                output_dim = output.shape[-1]
                output = output[1:].view(-1, output_dim)
                trg = trg[1:].view(-1)
                
                loss = criterion(output, trg)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
                optimizer.step()
                
                total_loss += loss.item()
            
            # Validation
            val_acc = evaluate(model, dev_loader, device)
            wandb.log({
                'epoch': epoch,
                'train_loss': total_loss/len(train_loader),
                'val_acc': val_acc
            })

def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for src, trg, src_lens, trg_lens in loader:
            src = src.to(device)
            trg = trg.to(device)
            
            outputs = model(src, src_lens, trg, 0)  # No teacher forcing
            outputs = outputs.argmax(dim=-1)
            
            # Calculate accuracy
            mask = (trg != 0)
            correct += ((outputs == trg) * mask).sum().item()
            total += mask.sum().item()
    
    return correct / total

# To start the sweep:
# wandb sweep config.yaml
# Then run agents with:
# wandb agent SWEEP_ID


In [14]:
train_loader, dev_loader, test_loader, src_vocab, trg_vocab = get_dataloaders()
print(f"Source vocab size: {len(src_vocab.char2idx)}")
print(f"Target vocab size: {len(trg_vocab.char2idx)}")
print(f"Training batches: {len(train_loader)}")

Source vocab size: 30
Target vocab size: 14
Training batches: 691


In [ ]:
# Initialize sweep
sweep_id = wandb.sweep(
    sweep=sweep_config,  # Your sweep configuration dictionary
    project="DA6401_A3",
    entity="megh_m-iit-madras"
)

# Run sweep agents
wandb.agent(sweep_id, function=train)

Create sweep with ID: vew3zh1e
Sweep URL: https://wandb.ai/megh_m-iit-madras/DA6401_A3/sweeps/vew3zh1e


wandb: Agent Starting Run: k3oiog37 with config:
wandb: 	batch_size: 64
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.3
wandb: 	embedding_size: 64
wandb: 	hidden_size: 256
wandb: 	learning_rate: 0.0001
wandb: 	num_layers: 1
